# Selecting a subset of time periods

Running a capacity expansion model for multiple regions with 8760 hours of load/generation data might be too computationally complex. PowerGenome includes a method (with more to come in the future) for selecting a representitive subset of periods. Doing so requires first generating all load and generation profiles.

The GenX capacity expansion model can also perform time reduction and the method has been updated more recently than what is used in PowerGenome. If you are using GenX it is better to set the parameter `reduce_time_domain` to `false` and use the TDR settings in GenX.

In [2]:
import warnings

warnings.simplefilter("ignore")

In [1]:
from pathlib import Path
import itertools

import pandas as pd
from powergenome.load_profiles import make_final_load_curves
from powergenome.generators import GeneratorClusters
from powergenome.util import (
    build_scenario_settings,
    init_pudl_connection,
    load_settings,
    reverse_dict_of_lists,
    check_settings
)

from powergenome.GenX import reduce_time_domain
from powergenome.external_data import make_generator_variability

## Import settings
This assumes that the settings file is set up for multiple scenarios/planning periods. If you are using a settings file with only a single scenario/planning period, remove or comment out the line with `build_scenario_settings`.

In [5]:
cwd = Path.cwd()

settings_path = (
    cwd.parent / "example_systems" / "ISONE" / "settings"
)
settings = load_settings(settings_path)
settings["input_folder"] = settings_path.parent / settings["input_folder"]
scenario_definitions = pd.read_csv(
    settings["input_folder"] / settings["scenario_definitions_fn"]
)
scenario_settings = build_scenario_settings(settings, scenario_definitions)

pudl_engine, pudl_out, pg_engine = init_pudl_connection(
    freq="AS",
    start_year=min(settings.get("eia_data_years")),
    end_year=max(settings.get("eia_data_years")),
)

check_settings(settings, pg_engine)

The parameter value 'sampled' from column 'time_series' in your scenario definitions file is not included in the 'settings_management' dictionary. Settings for case id 's1' will not be modified to reflect this scenario.


## Load curves

In [6]:
load_curves = make_final_load_curves(pg_engine, scenario_settings[2030]["p1"])
load_curves


            *****************************
            Regional load data sources have not been specified. Defaulting to EFS load data.
            Check your settings file, and please specify the preferred source for load data
            (FERC, EFS, USER) either for each region or for the entire system with the setting
            "regional_load_source".
            *****************************
            


region,NENGREST,NENG_CT,NENG_ME
time_index,,,
1,9215,3556,1097
2,8791,3405,1033
3,8422,3266,982
4,8144,3187,946
5,8011,3158,934
...,...,...,...
8756,11846,4793,1571
8757,12930,5192,1579
8758,12908,5268,1517


## Generation profiles

In [7]:
gc = GeneratorClusters(pudl_engine, pudl_out, pg_engine, scenario_settings[2030]["p1"])
all_gens = gc.create_all_generators()

/Users/gs5183/miniconda3/envs/powergenome/lib/python3.10/site-packages/geopandas/geodataframe.py:1443: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/Users/gs5183/Documents/PowerGenome/powergenome/resource_clusters.py:274: FutureWarning: 'ParquetDataset.schema' attribute is deprecated as of pyarrow 5.0.0 and will be removed in a future version. Specify 'use_legacy_dataset=False' while constructing the ParquetDataset, and then use the '.schema' attribute instead (which will return an Arrow schema instead of a Parquet schema).
  self._columns = self._dataset.schema.names


36.800000000000004  MW without lat/lon


********************************
When adding plant entity/boiler info to generators and filling missing seasonal capacity values, technologyNatural Gas Fired Combined Cycle changed capacity from 15950.7 to 16054.1
********************************
2023-06-13 06:19:09 [    INFO] catalystcoop.pudl.transform.eia861:456 Started with 325 missing BA Codes out of 12670 records (2.57%)
2023-06-13 06:19:10 [    INFO] catalystcoop.pudl.transform.eia861:480 Ended with 325 missing BA Codes out of 12670 records (2.57%)
2023-06-13 06:19:10 [    INFO] catalystcoop.pudl.output.eia860:177 97.4% of plant records have consistently reported BA Codes
2023-06-13 06:19:10 [    INFO] catalystcoop.pudl.output.eia860:227 Before any filling treatment has been applied. 2.6% of records have no BA codes
2023-06-13 06:19:10 [    INFO] catalystcoop.pudl.output.eia860:227 Backfilling and consistent value is the same. Filled w/ most consistent BA code. 2.6% of records have no BA codes
2023-06-13 06:19:10 [    INFO] cata

Creating gdf
['Other Natural Gas', 'Solar Photovoltaic', 'Batteries', 'Natural Gas Fired Combustion Turbine', 'Conventional Hydroelectric']
Creating gdf
['Other_peaker', 'Natural Gas Fired Combustion Turbine', 'Solar Photovoltaic', 'Other Natural Gas', 'Natural Gas Steam Turbine', 'Natural Gas Fired Combined Cycle', 'Conventional Hydroelectric', 'Batteries', 'Onshore Wind Turbine', 'Biomass', 'All Other']


No model tag values found for LDS ('LDS')
No model tag values found for Hydro_Energy_to_Power_Ratio ('Hydro_Energy_to_Power_Ratio')
No model tag values found for MinCapTag_2 ('MinCapTag_2')
No model tag values found for MinCapTag_3 ('MinCapTag_3')
No model tag values found for Reg_Max ('Reg_Max')
No model tag values found for Rsv_Max ('Rsv_Max')
/Users/gs5183/Documents/PowerGenome/powergenome/generators.py:3554: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.results["profile"][i] = clusters["profile"][0]
/Users/gs5183/Documents/PowerGenome/powergenome/generators.py:3554: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [8]:
gen_variability = make_generator_variability(all_gens)

## Reduce time domain
This function selects `N` periods of `x` days from the 8760 hours. It uses the settings parameters
- `reduce_time_domain` (a boolean value)
- `time_domain_periods` (`N`)
- `time_domain_days_per_period` (`x`)
- `include_peak_day` (if the day of peak demand should always be included in the output)
- `demand_weight_factor` (weighting factor for demand relative to generation profiles)

It outputs time reduced generation profiles and load profiles, along with a dataframe that tracks the sequential order of cluster slots in a year. The sequential order is needed to track long-duration storage across time. The weight of hours in each cluster are provided sequentially in the column `Sub_Weights`.

In [10]:
(
    reduced_resource_profile,
    reduced_load_profile,
    time_series_mapping,
    representative_point,
) = reduce_time_domain(gen_variability, load_curves, scenario_settings[2030]["p1"])

In [11]:
# Resource profiles are in the same column order as rows in all_gens
reduced_resource_profile

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77
Time_Index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1.0,1.0,0.4467,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.2949,1.0,0.4467,0.0,1.0,1.0,0.3409,1.0,1.0,1.0,1.0,1.0,0.1727,1.0,0.3409,0.0,1.0,1.0,0.6644,1.0,1.0,1.0,0.5246,1.0,0.6644,0.0,1.0,1.0,1.0,1.0,0.326510,0.761920,0.797867,1.000000,0.149718,0.886261,0.448254,0.814466,0.694686,0.892695,0.590493,0.513469,1.0,1.0,1.0,1.0,0.848894,0.870309,0.630747,0.585088,0.293596,0.512011,1.0,1.0,1.0,1.0,0.922187,0.956431,0.919378,0.936693,0.717576,0.820905,0.112689,0.104027,0.126638,0.317829,0.316278,0.296490
2,1.0,1.0,0.4465,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.2855,1.0,0.4465,0.0,1.0,1.0,0.3409,1.0,1.0,1.0,1.0,1.0,0.1644,1.0,0.3409,0.0,1.0,1.0,0.6646,1.0,1.0,1.0,0.5049,1.0,0.6646,0.0,1.0,1.0,1.0,1.0,0.262810,0.679032,0.791437,1.000000,0.111764,0.832097,0.510635,0.803070,0.726342,0.866608,0.660334,0.335984,1.0,1.0,1.0,1.0,0.858507,0.829902,0.600321,0.560315,0.266590,0.453836,1.0,1.0,1.0,1.0,0.895880,0.926753,0.862824,0.920390,0.654849,0.769258,0.043916,0.040452,0.049787,0.194500,0.192879,0.182809
3,1.0,1.0,0.4464,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.2822,1.0,0.4464,0.0,1.0,1.0,0.3409,1.0,1.0,1.0,1.0,1.0,0.2645,1.0,0.3409,0.0,1.0,1.0,0.6647,1.0,1.0,1.0,0.4734,1.0,0.6647,0.0,1.0,1.0,1.0,1.0,0.377515,0.800018,0.812051,0.846428,0.143284,0.516701,0.420613,0.637623,0.566849,0.702615,0.520600,0.182269,1.0,1.0,1.0,1.0,0.914197,0.995875,0.800432,0.756769,0.355582,0.634766,1.0,1.0,1.0,1.0,0.894452,0.893841,0.859792,0.899866,0.630715,0.745312,0.035556,0.032538,0.040722,0.082102,0.080851,0.077988
4,1.0,1.0,0.4462,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.2999,1.0,0.4462,0.0,1.0,1.0,0.3409,1.0,1.0,1.0,1.0,1.0,0.2140,1.0,0.3409,0.0,1.0,1.0,0.6648,1.0,1.0,1.0,0.4430,1.0,0.6648,0.0,1.0,1.0,1.0,1.0,0.339584,0.752027,0.789697,0.872623,0.125280,0.594472,0.393500,0.478903,0.434019,0.529905,0.374286,0.177643,1.0,1.0,1.0,1.0,0.880523,0.897995,0.710771,0.650341,0.295549,0.623322,1.0,1.0,1.0,1.0,0.865521,0.880251,0.820155,0.898597,0.562186,0.716275,0.024355,0.022301,0.028550,0.090504,0.089019,0.087735
5,1.0,1.0,0.4460,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.2623,1.0,0.4460,0.0,1.0,1.0,0.3409,1.0,1.0,1.0,1.0,1.0,0.1407,1.0,0.3409,0.0,1.0,1.0,0.6649,1.0,1.0,1.0,0.4305,1.0,0.6649,0.0,1.0,1.0,1.0,1.0,0.246250,0.641637,0.760396,0.917902,0.071759,0.600711,0.359101,0.558137,0.482576,0.618053,0.358666,0.263142,1.0,1.0,1.0,1.0,0.677017,0.633942,0.435773,0.432344,0.191068,0.391429,1.0,1.0,1.0,1.0,0.827164,0.822829,0.774091,0.860527,0.500022,0.673711,0.004485,0.004101,0.005387,0.066136,0.064974,0.065513
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8756,1.0,1.0,0.4476,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.1702,1.0,0.4476,0.0,1.0,1.0,0.3409,1.0,1.0,1.0,1.0,1.0,0.1340,1.0,0.3409,0.0,1.0,1.0,0.6638,1.0,1.0,1.0,0.2243,1.0,0.6638,0.0,1.0,1.0,1.0,1.0,0.228918,0.401395,0.225927,0.143577,0.092088,0.055788,0.190230,0.310401,0.115799,0.414936,0.311953,0.972474,1.0,1.0,1.0,1.0,0.487207,0.793695,0.424499,0.387240,0.228279,0.271431,1.0,1.0,1.0,1.0,0.687864,0.591491,0.598528,0.356457,0.463492,0.307882,0.637675,0.584121,0.759315,0.632499,0.630755,0.633374
8757,1.0,1.0,0.4474,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.1729,1.0,0.4474,0.0,1.0,1.0,0.3409,1.0,1.0,1.0,1.0,1.0,0.1134,1.0,0.3409,0.0,1.0,1.0,0.6639,1.0,1.0,1.0,0.1976,1.0,0.6639,0.0,1.0,1.0,1.0,1.0,0.204692,0.407841,0.333962,0.455153,0.051496,0.325811,0.344991,0.261895,0.207955,0.314317,0.317417,1.000000,1.0,1.0,1.0,1.0,0.387605,0.679635,0.386457,0.342486,0.165261,0.194961,1.0,1.0,1.0,1.0,0.682108,0.564835,0.566073,0.487513